In [7]:
import requests
import json
from collections import defaultdict


#中英對照
name_map = {
    "Temperature": "溫度",
    "DewPoint": "露點溫度",
    "RelativeHumidity": "相對濕度",
    "ApparentTemperature": "體感溫度",
    "ComfortIndex": "舒適度",
    "ComfortIndexDescription": "舒適度描述",
    "WindSpeed": "風速",
    "BeaufortScale": "蒲福風級",
    "ProbabilityOfPrecipitation": "降雨機率",
    "Weather": "天氣現象",
    "WeatherDescription": "天氣描述",
    "WindDirection": "風向",
    "WeatherCode": "天氣代碼"
}

In [8]:
def translate_name(be_translated_name):
    ch_name = name_map.get(be_translated_name, be_translated_name)
    return ch_name

url = f"https://opendata.cwa.gov.tw/api/v1/rest/datastore/F-D0047-069"
api_token = "CWA-8917E0F0-A194-4359-9634-D8AF9CDBB872"
def get_weather(city_name, district_name, show_top_ndata=10):
    params = {
        "Authorization" : api_token,
        "format": "JSON",
        "LocationName" : district_name
    }
    try:
        response = requests.get(url, params=params)
        response.raise_for_status()
        data = response.json()

        print("臺灣各縣市鄉鎮未來3天天氣預報")
        cities = data['records']['Locations']
        found_city = False
        found_district = False

        for city in cities:
            if city['LocationsName'] == city_name:
                found_city = True
                for district in city['Location']:
                    if district['LocationName'] == district_name:
                        found_district = True
                        print(f"\n📍 {city_name} {district_name}")

                        weather_elements = district['WeatherElement']
                        weather_merged = defaultdict(dict)

                        for elem in weather_elements:
                            elem_name = elem['ElementName']
                            for time_list in elem['Time'][:show_top_ndata]:
                                # 自動偵測是 DataTime 或 StartTime
                                time = time_list.get('DataTime') or time_list.get('StartTime')
                                element_value = time_list['ElementValue']
                                if element_value:
                                    for k, v in element_value[0].items():#.items()回傳(key, value)列表
                                        weather_merged[time][k] = v

                        # 印出前 N 筆資料
                        for time in sorted(weather_merged.keys())[:show_top_ndata]:
                            print(f"\n🕒 {time}")
                            for k, v in weather_merged[time].items():
                                print(f"  {translate_name(k)}: {v}")
                        return  
                             
        #if not found_district:
        print(f"⚠️ 找不到 {district} 的天氣資料。")

    except requests.exceptions.HTTPError as http_err:
        print("❌ HTTP 錯誤：", http_err)
    except requests.exceptions.RequestException as req_err:
        print("❌ 其他請求錯誤（例如網路錯誤）", req_err)
    except KeyError as ke:
        print("❌ 回傳資料格式錯誤，缺少欄位：", ke)
    except Exception as e:
        print("❌ 其他非 requests 錯誤：", e)  

In [9]:
if __name__ == "__main__":
    get_weather("新北市", "深坑區", show_top_ndata=10)

臺灣各縣市鄉鎮未來3天天氣預報

📍 新北市 深坑區

🕒 2025-06-19T06:00:00+08:00
  溫度: 25
  露點溫度: 23
  相對濕度: 91
  體感溫度: 29
  舒適度: 24
  舒適度描述: 舒適
  風速: 1
  蒲福風級: <= 1
  風向: 偏南風
  降雨機率: 0
  天氣現象: 晴
  天氣代碼: 01
  天氣描述: 晴。降雨機率0%。溫度攝氏25至27度。舒適。偏南風 平均風速<= 1級(每秒1公尺)。相對濕度82至91%。

🕒 2025-06-19T07:00:00+08:00
  溫度: 26
  露點溫度: 23
  相對濕度: 86
  體感溫度: 30
  舒適度: 25
  舒適度描述: 舒適

🕒 2025-06-19T08:00:00+08:00
  溫度: 27
  露點溫度: 24
  相對濕度: 82
  體感溫度: 32
  舒適度: 26
  舒適度描述: 舒適

🕒 2025-06-19T09:00:00+08:00
  溫度: 30
  露點溫度: 25
  相對濕度: 79
  體感溫度: 35
  舒適度: 28
  舒適度描述: 悶熱
  風速: 1
  蒲福風級: <= 1
  風向: 西南風
  降雨機率: 0
  天氣現象: 晴
  天氣代碼: 01
  天氣描述: 晴。降雨機率0%。溫度攝氏30至32度。悶熱。西南風 平均風速<= 1級(每秒1公尺)。相對濕度77至79%。

🕒 2025-06-19T10:00:00+08:00
  溫度: 31
  露點溫度: 27
  相對濕度: 77
  體感溫度: 37
  舒適度: 29
  舒適度描述: 悶熱

🕒 2025-06-19T11:00:00+08:00
  溫度: 32
  露點溫度: 28
  相對濕度: 77
  體感溫度: 39
  舒適度: 30
  舒適度描述: 悶熱

🕒 2025-06-19T12:00:00+08:00
  溫度: 33
  露點溫度: 29
  相對濕度: 79
  體感溫度: 40
  舒適度: 31
  舒適度描述: 易中暑
  風速: 1
  蒲福風級: <= 1
  風向: 東南風
  降雨機率: 20
  天氣現象: 晴
  天氣代碼: 01
  天氣描述